In [805]:
import os
import tarfile
import json
import pandas as pd
#from google.colab import drive

In [806]:
#Set WD
# Use this line in Colab
#drive.mount("/content/drive")
#Use this line in local jupyter
os.chdir("/Users/andreaprenner/Desktop/Master_UvA/1. Semester/FundamentalsOfDS/Assignment1")

In [807]:
# Use this line in Colab
#tar = tarfile.open('/content/drive/My Drive/Colab Notebooks/Assignment 1 FDS/geotagged_tweets_20160812-0912.tar')
# Use this line in local jupyter
tar = tarfile.open("geotagged_tweets_20160812-0912.tar.gz")
files = tar.getmembers()

In [808]:
f = tar.extractfile(files[0]) 
data = f.readlines()

In [796]:
#One tweet entry in raw data file
json.loads(data[0])

{'created_at': 'Fri Aug 12 10:04:00 +0000 2016',
 'id': 764039724818272256,
 'id_str': '764039724818272256',
 'text': '@theblaze @realDonaldTrump https://t.co/TY9DlZ584c',
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'truncated': False,
 'in_reply_to_status_id': 764038820476051456,
 'in_reply_to_status_id_str': '764038820476051456',
 'in_reply_to_user_id': 10774652,
 'in_reply_to_user_id_str': '10774652',
 'in_reply_to_screen_name': 'theblaze',
 'user': {'id': 366636488,
  'id_str': '366636488',
  'name': 'GIL DUPUY',
  'screen_name': 'DUPUY77',
  'location': 'Miami',
  'url': 'http://ggm-dupuy.com',
  'description': "Fashion photographer, love action and adventure, care for the less fortunate, don't tolerate any kind of racism regardless of race or religion",
  'protected': False,
  'verified': False,
  'followers_count': 186,
  'friends_count': 446,
  'listed_count': 19,
  'favourites_count': 1708,
  'statuses_count': 17620,
  'cre

In [830]:
#Change dictionary into dataframe
#Load dictionaries within dictionaries (user, place) separately. From user exclude unnecessary columns
data_formatted = []
for i in range(0, 100): 
    data_f = json.loads(data[i])
    user = data_f['user']
    user = pd.DataFrame([user])
    unwanted_user = set(['id_str', 'profile_background_color', 'profile_background_image_url', 
                         'profile_background_image_url_https', 'profile_background_tile', 
                        'profile_link_color', 'profile_sidebar_border_color', 
                        'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 
                        'profile_image_url', 'profile_image_url_https', 'default_profile_image']) 
    for unwanted_key in unwanted_user: del user[unwanted_key]
    place = data_f['place']
    place = pd.DataFrame([place])
    place = place[['full_name', 'country_code', 'bounding_box']]
    unwanted = set(['user', 'place', 'entities']) #Dictionaries within dictionaries
    for unwanted_key in unwanted: del data_f[unwanted_key]
    data_pre = pd.DataFrame([data_f])
    user = user.rename({'id':'user_id'}, axis=1)
    if i == 0: 
        data_formatted = data_pre
        user_formatted = user
        place_formatted = place
    else:
        data_formatted = data_formatted.append(data_pre, ignore_index=True)
        user_formatted = user_formatted.append(user, ignore_index=True)
        place_formatted = place_formatted.append(place, ignore_index=True)

In [831]:
#Merge data frame (without dictionaries within dictionaries), user and place dictionary together
data_formatted = pd.concat([data_formatted, user_formatted, place_formatted], axis=1)
data_formatted.head()

,contributors,coordinates,created_at,extended_entities,favorite_count,favorited,filter_level,geo,id,id_str,...,screen_name,statuses_count,time_zone,url,user_id,utc_offset,verified,full_name,country_code,bounding_box
0,None,None,Fri Aug 12 10:04:00 +0000 2016,"{'media': [{'id': 764039718237409281, 'id_str'...",0,False,low,None,764039724818272256,764039724818272256,...,DUPUY77,17620,None,http://ggm-dupuy.com,366636488,None,False,"Frontenac, MO",US,"{'type': 'Polygon', 'coordinates': [[[-90.4339..."
1,None,None,Fri Aug 12 10:04:02 +0000 2016,NaN,0,False,low,None,764039733076897792,764039733076897792,...,redoctapus,5046,Eastern Time (US & Canada),None,82496193,-14400,False,"Baton Rouge, LA",US,"{'type': 'Polygon', 'coordinates': [[[-91.2189..."
2,None,None,Fri Aug 12 10:04:10 +0000 2016,"{'media': [{'id': 764039762973896704, 'id_str'...",0,False,low,None,764039769244348417,764039769244348417,...,DUPUY77,17621,None,http://ggm-dupuy.com,366636488,None,False,"Frontenac, MO",US,"{'type': 'Polygon', 'coordinates': [[[-90.4339..."
3,None,None,Fri Aug 12 10:04:21 +0000 2016,NaN,0,False,low,None,764039812479225856,764039812479225856,...,jjorgi30,2231,Melbourne,None,44032624,36000,False,"Melbourne, Victoria",AU,"{'type': 'Polygon', 'coordinates': [[[144.5937..."
4,None,None,Fri Aug 12 10:04:30 +0000 2016,NaN,0,False,low,None,764039849850482689,764039849850482689,...,bspence5,92605,None,None,769208504,None,False,"Baltimore, MD",US,"{'type': 'Polygon', 'coordinates': [[[-76.7115..."


In [832]:
#Exclude tweets outside of US (location) - filter on country code US (Country where tweet is posted)
data_formatted = data_formatted[data_formatted['country_code'] == "US"]
#Excluding tweets referring to Clinton & Trump
#Getting sentiment score on daily basis 

In [ ]:
#Putting state from full_name column into separate column

In [833]:
#Determine tags and hashs (some tweets dont have tags only hash)
data_formatted['tags_mention'] = data_formatted.text.str.findall(r'(?<![@\w])@(\w{1,25})').apply(','.join)
data_formatted['hash_mention'] = data_formatted.text.str.findall(r"#(\w+)").apply(','.join)
data_formatted['tags_hash'] = data_formatted['hash_mention'] + data_formatted['tags_mention']

In [834]:
#Lower cases in tags and hashes
data_formatted['tags_hash'] = data_formatted['tags_hash'].apply(lambda x: x.lower())

In [835]:
#See if either trum or clinton mentioned in hash or tags
data_formatted['Trump'] = data_formatted['tags_hash'].str.contains(r'trump', na=False)
data_formatted['Clinton'] = data_formatted['tags_hash'].str.contains(r'clinton', na=False)

**SENTIMENT ANALYSIS**

In [837]:
import nltk
from nltk.corpus import stopwords
import random
import re

In [838]:
#Loading training dataset 
training_set = pd.read_csv("trainingandtestdata/training.1600000.processed.noemoticon.csv", 
                 encoding = 'latin-1',
                 names = ["Polarity", "Tweet ID","Date","Query","User","Text"])

In [839]:
#Define english stopwords
stopwords = stopwords.words('english')

In [840]:
#Filter out Polarity Score equal 0 or 4
training_set = training_set[(training_set["Polarity"] == 4) | (training_set["Polarity"] == 0)]

In [841]:
#Data cleaning - removing punctuation from text; removing tags; removing links
training_set['Text'] = training_set['Text'].str.replace('@[^\s]+','')
training_set['Text'] = training_set['Text'].str.replace('[^\w\s]','')
training_set['Text'] = training_set['Text'].str.replace('\[.*?\]','')
training_set['Text'] = training_set['Text'].str.replace('[‘’“”…]','')
training_set['Text'] = training_set['Text'].str.replace('\w*\d\w*','')

In [842]:
#Create data that puts tweet texts together with sentiment score
tweets_and_polarity_scores = []
for i in range(0, len(training_set)):
    if training_set["Polarity"].iloc[i] == 0 or training_set["Polarity"].iloc[i] == 4: 
        tweet_text = training_set['Text'].iloc[i]
        tweet_words_in_list = [e.lower() for e in tweet_text.split()]
        tweet_words_in_list_wo_stopwords = [e for e in tweet_words_in_list if not e in stopwords]
        tweet_polarity_score = training_set["Polarity"].iloc[i]
        tweets_and_polarity_scores.append((tweet_words_in_list_wo_stopwords, tweet_polarity_score))
    else: 
        continue

In [683]:
#Select random sample - otherwise the classifier will be too slow
random_sample = random.sample(tweets_and_polarity_scores, k = 10000)

In [684]:
def get_words_in_tweets(tweets):
    all_words= []
    for (words, sentiment) in tweets:
        all_words.extend(words)
    return all_words

def get_word_features(wordlist):
    wordlist = nltk.FreqDist(wordlist)
    word_features = list(wordlist.keys())
    return word_features

word_features = get_word_features(get_words_in_tweets(random_sample))

In [685]:
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features["contains(%s)" % word] = (word in document_words)
    return features

In [686]:
final_training_set = nltk.classify.apply_features(extract_features, random_sample)

In [687]:
#Classifier way too slow atm
classifier = nltk.NaiveBayesClassifier.train(final_training_set)

In [689]:
#Testing
tweet_positive = 'I like Larry'
classifier.classify(extract_features(tweet_positive.split()))
#classifier.show_most_informative_features(32)

4

In [857]:
#Loading test dataset 
test_set = pd.read_csv("trainingandtestdata/testdata.manual.2009.06.14.csv", 
                 encoding = 'latin-1',
                 names = ["Polarity", "Tweet ID","Date","Query","User","Text"])

In [859]:
#Data cleaning - removing punctuation from text
test_set['Text'] = test_set['Text'].str.replace('@[^\s]+','')
test_set['Text'] = test_set['Text'].str.replace('[^\w\s]','')
test_set['Text'] = test_set['Text'].str.replace('\[.*?\]','')
test_set['Text'] = test_set['Text'].str.replace('[‘’“”…]','')
test_set['Text'] = test_set['Text'].str.replace('\w*\d\w*','')

In [861]:
#Split words in tweets; remove stop words
test_set['Text'] = test_set['Text'].apply(lambda x: x.lower())
test_set['Text'] = test_set['Text'].apply(lambda x: x.split())
test_set['Text'] = test_set['Text'].apply(lambda x: [item for item in x if item not in stop])

In [863]:
#Apply classifier to column 
test_set['classification_polarity'] = test_set['Text'].apply(lambda x: classifier.classify(extract_features(x)))

In [865]:
test_set.head()

,Polarity,Tweet ID,Date,Query,User,Text,classification_polarity
0,4,3,Mon May 11 03:17:40 UTC 2009,kindle2,tpryan,"[loooooooovvvvvveee, dx, cool, fantastic, right]",4
1,4,4,Mon May 11 03:18:03 UTC 2009,kindle2,vcu451,"[reading, love, lee, childs, good, read]",4
2,4,5,Mon May 11 03:18:54 UTC 2009,kindle2,chadfu,"[ok, first, assesment, fucking, rocks]",4
3,4,6,Mon May 11 03:19:04 UTC 2009,kindle2,SIX15,"[youll, love, ive, mine, months, never, looked...",0
4,4,7,Mon May 11 03:21:41 UTC 2009,kindle2,yamarama,"[fair, enough, think, perfect]",4


**Topic Modelling - LDA**